Descrição do Modelo de Regressão

O modelo de regressão foi desenvolvido para prever o preço de casas com base em um conjunto de variáveis preditivas relacionadas às suas características estruturais e ao ambiente em que estão localizadas. Utilizando o Home Value Insights Dataset, o objetivo é identificar os fatores que mais influenciam o valor das propriedades e construir um modelo preditivo eficiente.
Características do Dataset

    Variáveis Preditivas:
    O conjunto de dados inclui os seguintes atributos relacionados às casas:
        Square_Footage: Tamanho da casa em pés quadrados. Casas maiores tendem a ter preços mais altos.
        Num_Bedrooms: Número de quartos. Geralmente, mais quartos aumentam o valor do imóvel.
        Num_Bathrooms: Número de banheiros. Casas com mais banheiros costumam ser mais valorizadas.
        Year_Built: Ano de construção da casa. Imóveis mais antigos podem ter menor valor devido ao desgaste.
        Lot_Size: Tamanho do terreno em acres. Lotes maiores frequentemente agregam valor ao imóvel.
        Garage_Size: Capacidade da garagem em número de carros. Garagens maiores aumentam o preço.
        Neighborhood_Quality: Qualidade do bairro, avaliada de 1 a 10. Bairros com melhor avaliação geralmente possuem casas mais caras.

    Variável Alvo:
        House_Price: O preço da casa, representado como uma variável contínua. Esse é o valor que o modelo busca prever.

Objetivo do Modelo

O objetivo é prever o preço de uma casa a partir de suas características, fornecendo insights sobre a relação entre as variáveis preditivas e o valor do imóvel. Além disso, o modelo pode ajudar a identificar os fatores mais influentes no preço, permitindo análise e interpretação de mercado.

In [ ]:
import numpy as np
import pandas as pd
import time

# ==================== Classe Regressão Neural ====================
class Regressao:
    def __init__(self, entradas_tamanho, camadas_ocultas, saidas_tamanho):
        # Define as dimensões das camadas da rede neural
        self.tamanhos_camadas = [entradas_tamanho] + camadas_ocultas + [saidas_tamanho]
        self.num_camadas = len(self.tamanhos_camadas) - 1

        # Inicializa pesos e vieses aleatoriamente
        self.pesos = [
            np.random.randn(self.tamanhos_camadas[i], self.tamanhos_camadas[i + 1]) * np.sqrt(2. / self.tamanhos_camadas[i])
            for i in range(self.num_camadas)
        ]
        self.vieses = [
            np.zeros((1, self.tamanhos_camadas[i + 1]))
            for i in range(self.num_camadas)
        ]
    
    # Função de ativação ReLU (Retifica valores negativos para zero)
    def relu(self, x):
        return np.maximum(0, x)

    # Derivada da função ReLU
    def relu_derivada(self, x):
        return (x > 0).astype(int)

    # Propagação para frente (cálculo das saídas)
    def propagacao_frente(self, entradas):
        self.ativacoes = [entradas]
        self.valores_z = []

        for i in range(self.num_camadas):
            z = np.dot(self.ativacoes[-1], self.pesos[i]) + self.vieses[i]
            self.valores_z.append(z)

            if i == self.num_camadas - 1:  # Camada de saída
                a = z  # Função identidade
            else:
                a = self.relu(z)
            self.ativacoes.append(a)
        return self.ativacoes[-1]

    # Função de perda: Erro Quadrático Médio (MSE)
    def calcular_mse(self, previsoes, verdadeiros):
        return np.mean((previsoes - verdadeiros) ** 2)

    # Cálculo do RMSE (raiz do MSE)
    def calcular_rmse(self, previsoes, verdadeiros):
        mse = self.calcular_mse(previsoes, verdadeiros)
        return np.sqrt(mse)

    # Retropropagação para ajustar pesos e vieses
    def retropropagacao(self, verdadeiros):
        m = verdadeiros.shape[0]
        verdadeiros = verdadeiros.reshape(-1, 1)

        gradientes_ativacoes = [(self.ativacoes[-1] - verdadeiros) / m]

        for i in reversed(range(self.num_camadas)):
            dz = (
                gradientes_ativacoes[0]
                if i == self.num_camadas - 1
                else gradientes_ativacoes[0] * self.relu_derivada(self.valores_z[i])
            )
            dw = np.dot(self.ativacoes[i].T, dz)
            db = np.sum(dz, axis=0, keepdims=True)

            dw += 2 * 0.01 * self.pesos[i]  # Regularização L2

            if i > 0:
                gradientes_ativacoes.insert(0, np.dot(dz, self.pesos[i].T))

            self.pesos[i] -= self.taxa_aprendizado * dw
            self.vieses[i] -= self.taxa_aprendizado * db

    # Treinamento do modelo
    def treinar(self, entradas, verdadeiros, epocas, taxa_aprendizado):
        self.taxa_aprendizado = taxa_aprendizado
        for epoca in range(epocas):
            previsoes = self.propagacao_frente(entradas)
            rmse = self.calcular_rmse(previsoes, verdadeiros)
            erro_percentual_medio = np.mean(np.abs(previsoes - verdadeiros) / np.maximum(np.abs(verdadeiros), 1e-10)) * 100

            self.retropropagacao(verdadeiros)

            if epoca % 10 == 0 or epoca == epocas - 1:
                print(f"Época {epoca} - RMSE: {rmse:.4f}, Erro percentual médio: {erro_percentual_medio:.2f}%")

    # Avaliação do modelo
    def avaliar(self, entradas, verdadeiros):
        previsoes = self.propagacao_frente(entradas)
        rmse = self.calcular_rmse(previsoes, verdadeiros)
        erro_percentual_medio = np.mean(np.abs(previsoes - verdadeiros) / np.maximum(np.abs(verdadeiros), 1e-10)) * 100
        print(f"RMSE: {rmse:.4f}")
        print(f"Erro percentual médio: {erro_percentual_medio:.2f}%")
        return rmse, erro_percentual_medio


In [ ]:

# ==================== Normalização de Dados ====================
def normalizar_dados(entradas):
    desvio_padrao = entradas.std(axis=0)
    desvio_padrao[desvio_padrao == 0] = 1
    return (entradas - entradas.mean(axis=0)) / desvio_padrao


In [ ]:

# ==================== Carregamento e Execução ====================
caminho_treino = './data/train_student_performance.csv'
caminho_teste = './data/test_student_performance.csv'

# Carregar dados
dados_treino = pd.read_csv(caminho_treino)
X_treino = normalizar_dados(dados_treino.iloc[:, :-1].values)
y_treino = dados_treino.iloc[:, -1].values

dados_teste = pd.read_csv(caminho_teste)
X_teste = normalizar_dados(dados_teste.iloc[:, :-1].values)
y_teste = dados_teste.iloc[:, -1].values




In [ ]:
#==================== Execução ====================
# Configuração da rede
rede = Regressao(entradas_tamanho=X_treino.shape[1], camadas_ocultas=[3, 6, 9, 3, 6, 9], saidas_tamanho=1)

# Treinamento
rede.treinar(X_treino, y_treino, epocas=10000, taxa_aprendizado=0.0001)

# Avaliação
rede.avaliar(X_teste, y_teste)

vou manter aqui caso a execução do google colab demore demais...

        Época 0 - RMSE: 666059.7819, Erro percentual médio: 100.00%
        Época 10 - RMSE: 667595.3863, Erro percentual médio: 100.34%
        Época 20 - RMSE: 667021.9296, Erro percentual médio: 100.21%
        Época 30 - RMSE: 666449.1267, Erro percentual médio: 100.09%
        Época 40 - RMSE: 665876.9771, Erro percentual médio: 99.96%
        Época 50 - RMSE: 665305.4801, Erro percentual médio: 99.83%
        Época 60 - RMSE: 664734.6352, Erro percentual médio: 99.71%
        Época 70 - RMSE: 664164.4417, Erro percentual médio: 99.58%
        Época 80 - RMSE: 663594.8991, Erro percentual médio: 99.46%
        Época 90 - RMSE: 663026.0067, Erro percentual médio: 99.33%
        Época 100 - RMSE: 662457.7639, Erro percentual médio: 99.21%
        Época 110 - RMSE: 661890.1701, Erro percentual médio: 99.08%
        Época 120 - RMSE: 661323.2247, Erro percentual médio: 98.96%
        Época 130 - RMSE: 660756.9272, Erro percentual médio: 98.83%
        Época 140 - RMSE: 660191.2769, Erro percentual médio: 98.71%
        Época 150 - RMSE: 659626.2732, Erro percentual médio: 98.58%
        Época 160 - RMSE: 659061.9155, Erro percentual médio: 98.46%
        Época 170 - RMSE: 658498.2033, Erro percentual médio: 98.33%
        Época 180 - RMSE: 657935.1358, Erro percentual médio: 98.21%
        Época 190 - RMSE: 657372.7126, Erro percentual médio: 98.08%
        Época 200 - RMSE: 656810.9330, Erro percentual médio: 97.96%
        Época 210 - RMSE: 656249.7964, Erro percentual médio: 97.84%
        Época 220 - RMSE: 655689.3022, Erro percentual médio: 97.71%
        Época 230 - RMSE: 655129.4499, Erro percentual médio: 97.59%
        Época 240 - RMSE: 654570.2388, Erro percentual médio: 97.46%
        Época 250 - RMSE: 654011.6684, Erro percentual médio: 97.34%
        Época 260 - RMSE: 653453.7380, Erro percentual médio: 97.22%
        Época 270 - RMSE: 652896.4470, Erro percentual médio: 97.09%
        Época 280 - RMSE: 652339.7949, Erro percentual médio: 96.97%
        Época 290 - RMSE: 651783.7811, Erro percentual médio: 96.85%
        Época 300 - RMSE: 651228.4050, Erro percentual médio: 96.73%
        Época 310 - RMSE: 650673.6659, Erro percentual médio: 96.60%
        Época 320 - RMSE: 650119.5633, Erro percentual médio: 96.48%
        Época 330 - RMSE: 649566.0966, Erro percentual médio: 96.36%
        Época 340 - RMSE: 649013.2653, Erro percentual médio: 96.24%
        Época 350 - RMSE: 648461.0686, Erro percentual médio: 96.11%
        Época 360 - RMSE: 647909.5061, Erro percentual médio: 95.99%
        Época 370 - RMSE: 647358.5771, Erro percentual médio: 95.87%
        Época 380 - RMSE: 646808.2811, Erro percentual médio: 95.75%
        Época 390 - RMSE: 646258.6175, Erro percentual médio: 95.63%
        Época 400 - RMSE: 645709.5856, Erro percentual médio: 95.50%
        Época 410 - RMSE: 645161.1849, Erro percentual médio: 95.38%
        Época 420 - RMSE: 644613.4149, Erro percentual médio: 95.26%
        Época 430 - RMSE: 644066.2748, Erro percentual médio: 95.14%
        Época 440 - RMSE: 643519.7642, Erro percentual médio: 95.02%
        Época 450 - RMSE: 642973.8825, Erro percentual médio: 94.90%
        Época 460 - RMSE: 642428.6290, Erro percentual médio: 94.78%
        Época 470 - RMSE: 641884.0032, Erro percentual médio: 94.66%
        Época 480 - RMSE: 641340.0045, Erro percentual médio: 94.53%
        Época 490 - RMSE: 640796.6324, Erro percentual médio: 94.41%
        Época 500 - RMSE: 640253.8862, Erro percentual médio: 94.29%
        Época 510 - RMSE: 639711.7653, Erro percentual médio: 94.17%
        Época 520 - RMSE: 639170.2693, Erro percentual médio: 94.05%
        Época 530 - RMSE: 638629.3974, Erro percentual médio: 93.93%
        Época 540 - RMSE: 638089.1492, Erro percentual médio: 93.81%
        Época 550 - RMSE: 637549.5240, Erro percentual médio: 93.69%
        Época 560 - RMSE: 637010.5213, Erro percentual médio: 93.57%
        Época 570 - RMSE: 636472.1404, Erro percentual médio: 93.45%
        Época 580 - RMSE: 635934.3809, Erro percentual médio: 93.33%
        Época 590 - RMSE: 635397.2421, Erro percentual médio: 93.22%
        Época 600 - RMSE: 634860.7235, Erro percentual médio: 93.10%
        Época 610 - RMSE: 634324.8244, Erro percentual médio: 92.98%
        Época 620 - RMSE: 633789.5444, Erro percentual médio: 92.86%
        Época 630 - RMSE: 633254.8828, Erro percentual médio: 92.74%
        Época 640 - RMSE: 632720.8391, Erro percentual médio: 92.62%
        Época 650 - RMSE: 632187.4126, Erro percentual médio: 92.50%
        Época 660 - RMSE: 631654.6029, Erro percentual médio: 92.38%
        Época 670 - RMSE: 631122.4093, Erro percentual médio: 92.27%
        Época 680 - RMSE: 630590.8313, Erro percentual médio: 92.15%
        Época 690 - RMSE: 630059.8683, Erro percentual médio: 92.03%
        Época 700 - RMSE: 629529.5197, Erro percentual médio: 91.91%
        Época 710 - RMSE: 628999.7850, Erro percentual médio: 91.79%
        Época 720 - RMSE: 628470.6635, Erro percentual médio: 91.67%
        Época 730 - RMSE: 627942.1548, Erro percentual médio: 91.56%
        Época 740 - RMSE: 627414.2583, Erro percentual médio: 91.44%
        Época 750 - RMSE: 626886.9733, Erro percentual médio: 91.32%
        Época 760 - RMSE: 626360.2993, Erro percentual médio: 91.20%
        Época 770 - RMSE: 625834.2358, Erro percentual médio: 91.09%
        Época 780 - RMSE: 625308.7822, Erro percentual médio: 90.97%
        Época 790 - RMSE: 624783.9379, Erro percentual médio: 90.85%
        Época 800 - RMSE: 624259.7023, Erro percentual médio: 90.74%
        Época 810 - RMSE: 623736.0750, Erro percentual médio: 90.62%
        Época 820 - RMSE: 623213.0552, Erro percentual médio: 90.50%
        Época 830 - RMSE: 622690.6426, Erro percentual médio: 90.39%
        Época 840 - RMSE: 622168.8364, Erro percentual médio: 90.27%
        Época 850 - RMSE: 621647.6361, Erro percentual médio: 90.15%
        Época 860 - RMSE: 621127.0412, Erro percentual médio: 90.04%
        Época 870 - RMSE: 620607.0512, Erro percentual médio: 89.92%
        Época 880 - RMSE: 620087.6653, Erro percentual médio: 89.81%
        Época 890 - RMSE: 619568.8832, Erro percentual médio: 89.69%
        Época 900 - RMSE: 619050.7042, Erro percentual médio: 89.57%
        Época 910 - RMSE: 618533.1278, Erro percentual médio: 89.46%
        Época 920 - RMSE: 618016.1533, Erro percentual médio: 89.34%
        Época 930 - RMSE: 617499.7803, Erro percentual médio: 89.23%
        Época 940 - RMSE: 616984.0082, Erro percentual médio: 89.11%
        Época 950 - RMSE: 616468.8365, Erro percentual médio: 89.00%
        Época 960 - RMSE: 615954.2645, Erro percentual médio: 88.88%
        Época 970 - RMSE: 615440.2917, Erro percentual médio: 88.77%
        Época 980 - RMSE: 614926.9176, Erro percentual médio: 88.65%
        Época 990 - RMSE: 614414.1417, Erro percentual médio: 88.54%
        Época 1000 - RMSE: 613901.9632, Erro percentual médio: 88.42%
        Época 1010 - RMSE: 613390.3818, Erro percentual médio: 88.31%
        Época 1020 - RMSE: 612879.3968, Erro percentual médio: 88.20%
        Época 1030 - RMSE: 612369.0077, Erro percentual médio: 88.08%
        Época 1040 - RMSE: 611859.2139, Erro percentual médio: 87.97%
        Época 1050 - RMSE: 611350.0149, Erro percentual médio: 87.85%
        Época 1060 - RMSE: 610841.4102, Erro percentual médio: 87.74%
        Época 1070 - RMSE: 610333.3991, Erro percentual médio: 87.63%
        Época 1080 - RMSE: 609825.9812, Erro percentual médio: 87.51%
        Época 1090 - RMSE: 609319.1558, Erro percentual médio: 87.40%
        Época 1100 - RMSE: 608812.9225, Erro percentual médio: 87.28%
        Época 1110 - RMSE: 608307.2807, Erro percentual médio: 87.17%
        Época 1120 - RMSE: 607802.2298, Erro percentual médio: 87.06%
        Época 1130 - RMSE: 607297.7692, Erro percentual médio: 86.95%
        Época 1140 - RMSE: 606793.8985, Erro percentual médio: 86.83%
        Época 1150 - RMSE: 606290.6171, Erro percentual médio: 86.72%
        Época 1160 - RMSE: 605787.9244, Erro percentual médio: 86.61%
        Época 1170 - RMSE: 605285.8200, Erro percentual médio: 86.49%
        Época 1180 - RMSE: 604784.3031, Erro percentual médio: 86.38%
        Época 1190 - RMSE: 604283.3734, Erro percentual médio: 86.27%
        Época 1200 - RMSE: 603783.0302, Erro percentual médio: 86.16%
        Época 1210 - RMSE: 603283.2731, Erro percentual médio: 86.04%
        Época 1220 - RMSE: 602784.1014, Erro percentual médio: 85.93%
        Época 1230 - RMSE: 602285.5146, Erro percentual médio: 85.82%
        Época 1240 - RMSE: 601787.5122, Erro percentual médio: 85.71%
        Época 1250 - RMSE: 601290.0936, Erro percentual médio: 85.60%
        Época 1260 - RMSE: 600793.2584, Erro percentual médio: 85.49%
        Época 1270 - RMSE: 600297.0059, Erro percentual médio: 85.37%
        Época 1280 - RMSE: 599801.3356, Erro percentual médio: 85.26%
        Época 1290 - RMSE: 599306.2470, Erro percentual médio: 85.15%
        Época 1300 - RMSE: 598811.7395, Erro percentual médio: 85.04%
        Época 1310 - RMSE: 598317.8126, Erro percentual médio: 84.93%
        Época 1320 - RMSE: 597824.4657, Erro percentual médio: 84.82%
        Época 1330 - RMSE: 597331.6984, Erro percentual médio: 84.71%
        Época 1340 - RMSE: 596839.5100, Erro percentual médio: 84.60%
        Época 1350 - RMSE: 596347.9001, Erro percentual médio: 84.49%
        Época 1360 - RMSE: 595856.8681, Erro percentual médio: 84.38%
        Época 1370 - RMSE: 595366.4134, Erro percentual médio: 84.27%
        Época 1380 - RMSE: 594876.5356, Erro percentual médio: 84.16%
        Época 1390 - RMSE: 594387.2340, Erro percentual médio: 84.04%
        Época 1400 - RMSE: 593898.5083, Erro percentual médio: 83.93%
        Época 1410 - RMSE: 593410.3577, Erro percentual médio: 83.82%
        Época 1420 - RMSE: 592922.7818, Erro percentual médio: 83.72%
        Época 1430 - RMSE: 592435.7801, Erro percentual médio: 83.61%
        Época 1440 - RMSE: 591949.3520, Erro percentual médio: 83.50%
        Época 1450 - RMSE: 591463.4970, Erro percentual médio: 83.39%
        Época 1460 - RMSE: 590978.2145, Erro percentual médio: 83.28%
        Época 1470 - RMSE: 590493.5040, Erro percentual médio: 83.17%
        Época 1480 - RMSE: 590009.3651, Erro percentual médio: 83.06%
        Época 1490 - RMSE: 589525.7971, Erro percentual médio: 82.95%
        Época 1500 - RMSE: 589042.7995, Erro percentual médio: 82.84%
        Época 1510 - RMSE: 588560.3718, Erro percentual médio: 82.73%
        Época 1520 - RMSE: 588078.5135, Erro percentual médio: 82.62%
        Época 1530 - RMSE: 587597.2240, Erro percentual médio: 82.51%
        Época 1540 - RMSE: 587116.5029, Erro percentual médio: 82.41%
        Época 1550 - RMSE: 586636.3495, Erro percentual médio: 82.30%
        Época 1560 - RMSE: 586156.7634, Erro percentual médio: 82.19%
        Época 1570 - RMSE: 585677.7439, Erro percentual médio: 82.08%
        Época 1580 - RMSE: 585199.2907, Erro percentual médio: 81.97%
        Época 1590 - RMSE: 584721.4032, Erro percentual médio: 81.86%
        Época 1600 - RMSE: 584244.0808, Erro percentual médio: 81.76%
        Época 1610 - RMSE: 583767.3231, Erro percentual médio: 81.65%
        Época 1620 - RMSE: 583291.1294, Erro percentual médio: 81.54%
        Época 1630 - RMSE: 582815.4993, Erro percentual médio: 81.43%
        Época 1640 - RMSE: 582340.4322, Erro percentual médio: 81.33%
        Época 1650 - RMSE: 581865.9277, Erro percentual médio: 81.22%
        Época 1660 - RMSE: 581391.9852, Erro percentual médio: 81.11%
        Época 1670 - RMSE: 580918.6041, Erro percentual médio: 81.00%
        Época 1680 - RMSE: 580445.7840, Erro percentual médio: 80.90%
        Época 1690 - RMSE: 579973.5244, Erro percentual médio: 80.79%
        Época 1700 - RMSE: 579501.8246, Erro percentual médio: 80.68%
        Época 1710 - RMSE: 579030.6842, Erro percentual médio: 80.58%
        Época 1720 - RMSE: 578560.1028, Erro percentual médio: 80.47%
        Época 1730 - RMSE: 578090.0796, Erro percentual médio: 80.36%
        Época 1740 - RMSE: 577620.6143, Erro percentual médio: 80.26%
        Época 1750 - RMSE: 577151.7063, Erro percentual médio: 80.15%
        Época 1760 - RMSE: 576683.3551, Erro percentual médio: 80.05%
        Época 1770 - RMSE: 576215.5602, Erro percentual médio: 79.94%
        Época 1780 - RMSE: 575748.3210, Erro percentual médio: 79.83%
        Época 1790 - RMSE: 575281.6371, Erro percentual médio: 79.73%
        Época 1800 - RMSE: 574815.5079, Erro percentual médio: 79.62%
        Época 1810 - RMSE: 574349.9329, Erro percentual médio: 79.52%
        Época 1820 - RMSE: 573884.9115, Erro percentual médio: 79.41%
        Época 1830 - RMSE: 573420.4434, Erro percentual médio: 79.31%
        Época 1840 - RMSE: 572956.5279, Erro percentual médio: 79.20%
        Época 1850 - RMSE: 572493.1646, Erro percentual médio: 79.09%
        Época 1860 - RMSE: 572030.3529, Erro percentual médio: 78.99%
        Época 1870 - RMSE: 571568.0923, Erro percentual médio: 78.88%
        Época 1880 - RMSE: 571106.3823, Erro percentual médio: 78.78%
        Época 1890 - RMSE: 570645.2225, Erro percentual médio: 78.67%
        Época 1900 - RMSE: 570184.6122, Erro percentual médio: 78.57%
        Época 1910 - RMSE: 569724.5509, Erro percentual médio: 78.47%
        Época 1920 - RMSE: 569265.0382, Erro percentual médio: 78.36%
        Época 1930 - RMSE: 568806.0736, Erro percentual médio: 78.26%
        Época 1940 - RMSE: 568347.6565, Erro percentual médio: 78.15%
        Época 1950 - RMSE: 567889.7864, Erro percentual médio: 78.05%
        Época 1960 - RMSE: 567432.4629, Erro percentual médio: 77.94%
        Época 1970 - RMSE: 566975.6853, Erro percentual médio: 77.84%
        Época 1980 - RMSE: 566519.4533, Erro percentual médio: 77.74%
        Época 1990 - RMSE: 566063.7662, Erro percentual médio: 77.63%
        Época 2000 - RMSE: 565608.6236, Erro percentual médio: 77.53%
        Época 2010 - RMSE: 565154.0250, Erro percentual médio: 77.43%
        Época 2020 - RMSE: 564699.9698, Erro percentual médio: 77.32%
        Época 2030 - RMSE: 564246.4576, Erro percentual médio: 77.22%
        Época 2040 - RMSE: 563793.4879, Erro percentual médio: 77.12%
        Época 2050 - RMSE: 563341.0601, Erro percentual médio: 77.01%
        Época 2060 - RMSE: 562889.1738, Erro percentual médio: 76.91%
        Época 2070 - RMSE: 562437.8284, Erro percentual médio: 76.81%
        Época 2080 - RMSE: 561987.0234, Erro percentual médio: 76.71%
        Época 2090 - RMSE: 561536.7583, Erro percentual médio: 76.61%
        Época 2100 - RMSE: 561087.0327, Erro percentual médio: 76.51%
        Época 2110 - RMSE: 560637.8460, Erro percentual médio: 76.41%
        Época 2120 - RMSE: 560189.1977, Erro percentual médio: 76.31%
        Época 2130 - RMSE: 559741.0874, Erro percentual médio: 76.21%
        Época 2140 - RMSE: 559293.5144, Erro percentual médio: 76.11%
        Época 2150 - RMSE: 558846.4784, Erro percentual médio: 76.01%
        Época 2160 - RMSE: 558399.9788, Erro percentual médio: 75.91%
        Época 2170 - RMSE: 557954.0151, Erro percentual médio: 75.81%
        Época 2180 - RMSE: 557508.5868, Erro percentual médio: 75.71%
        Época 2190 - RMSE: 557063.6934, Erro percentual médio: 75.61%
        Época 2200 - RMSE: 556619.3344, Erro percentual médio: 75.51%
        Época 2210 - RMSE: 556175.5094, Erro percentual médio: 75.41%
        Época 2220 - RMSE: 555732.2178, Erro percentual médio: 75.31%
        Época 2230 - RMSE: 555289.4590, Erro percentual médio: 75.22%
        Época 2240 - RMSE: 554847.2327, Erro percentual médio: 75.12%
        Época 2250 - RMSE: 554405.5384, Erro percentual médio: 75.02%
        Época 2260 - RMSE: 553964.3754, Erro percentual médio: 74.92%
        Época 2270 - RMSE: 553523.7434, Erro percentual médio: 74.82%
        Época 2280 - RMSE: 553083.6418, Erro percentual médio: 74.73%
        Época 2290 - RMSE: 552644.0702, Erro percentual médio: 74.63%
        Época 2300 - RMSE: 552205.0280, Erro percentual médio: 74.53%
        Época 2310 - RMSE: 551766.5148, Erro percentual médio: 74.43%
        Época 2320 - RMSE: 551328.5300, Erro percentual médio: 74.34%
        Época 2330 - RMSE: 550891.0732, Erro percentual médio: 74.24%
        Época 2340 - RMSE: 550454.1438, Erro percentual médio: 74.14%
        Época 2350 - RMSE: 550017.7414, Erro percentual médio: 74.05%
        Época 2360 - RMSE: 549581.8655, Erro percentual médio: 73.95%
        Época 2370 - RMSE: 549146.5156, Erro percentual médio: 73.85%
        Época 2380 - RMSE: 548711.6912, Erro percentual médio: 73.76%
        Época 2390 - RMSE: 548277.3918, Erro percentual médio: 73.66%
        Época 2400 - RMSE: 547843.6169, Erro percentual médio: 73.56%
        Época 2410 - RMSE: 547410.3661, Erro percentual médio: 73.47%
        Época 2420 - RMSE: 546977.6388, Erro percentual médio: 73.37%
        Época 2430 - RMSE: 546545.4345, Erro percentual médio: 73.27%
        Época 2440 - RMSE: 546113.7528, Erro percentual médio: 73.18%
        Época 2450 - RMSE: 545682.5932, Erro percentual médio: 73.08%
        Época 2460 - RMSE: 545251.9551, Erro percentual médio: 72.99%
        Época 2470 - RMSE: 544821.8381, Erro percentual médio: 72.89%
        Época 2480 - RMSE: 544392.2418, Erro percentual médio: 72.80%
        Época 2490 - RMSE: 543963.1655, Erro percentual médio: 72.70%
        Época 2500 - RMSE: 543534.6089, Erro percentual médio: 72.60%
        Época 2510 - RMSE: 543106.5714, Erro percentual médio: 72.51%
        Época 2520 - RMSE: 542679.0526, Erro percentual médio: 72.41%
        Época 2530 - RMSE: 542252.0520, Erro percentual médio: 72.32%
        Época 2540 - RMSE: 541825.5691, Erro percentual médio: 72.22%
        Época 2550 - RMSE: 541399.6033, Erro percentual médio: 72.13%
        Época 2560 - RMSE: 540974.1543, Erro percentual médio: 72.03%
        Época 2570 - RMSE: 540549.2215, Erro percentual médio: 71.94%
        Época 2580 - RMSE: 540124.8045, Erro percentual médio: 71.84%
        Época 2590 - RMSE: 539700.9027, Erro percentual médio: 71.75%
        Época 2600 - RMSE: 539277.5157, Erro percentual médio: 71.66%
        Época 2610 - RMSE: 538854.6431, Erro percentual médio: 71.56%
        Época 2620 - RMSE: 538432.2842, Erro percentual médio: 71.47%
        Época 2630 - RMSE: 538010.4387, Erro percentual médio: 71.37%
        Época 2640 - RMSE: 537589.1061, Erro percentual médio: 71.28%
        Época 2650 - RMSE: 537168.2858, Erro percentual médio: 71.19%
        Época 2660 - RMSE: 536747.9774, Erro percentual médio: 71.09%
        Época 2670 - RMSE: 536328.1805, Erro percentual médio: 71.00%
        Época 2680 - RMSE: 535908.8945, Erro percentual médio: 70.91%
        Época 2690 - RMSE: 535490.1189, Erro percentual médio: 70.81%
        Época 2700 - RMSE: 535071.8534, Erro percentual médio: 70.72%
        Época 2710 - RMSE: 534654.0973, Erro percentual médio: 70.63%
        Época 2720 - RMSE: 534236.8502, Erro percentual médio: 70.54%
        Época 2730 - RMSE: 533820.1118, Erro percentual médio: 70.44%
        Época 2740 - RMSE: 533403.8813, Erro percentual médio: 70.35%
        Época 2750 - RMSE: 532988.1585, Erro percentual médio: 70.26%
        Época 2760 - RMSE: 532572.9428, Erro percentual médio: 70.17%
        Época 2770 - RMSE: 532158.2337, Erro percentual médio: 70.08%
        Época 2780 - RMSE: 531744.0308, Erro percentual médio: 69.99%
        Época 2790 - RMSE: 531330.3336, Erro percentual médio: 69.90%
        Época 2800 - RMSE: 530917.1416, Erro percentual médio: 69.81%
        Época 2810 - RMSE: 530504.4544, Erro percentual médio: 69.72%
        Época 2820 - RMSE: 530092.2714, Erro percentual médio: 69.63%
        Época 2830 - RMSE: 529680.5922, Erro percentual médio: 69.54%
        Época 2840 - RMSE: 529269.4163, Erro percentual médio: 69.45%
        Época 2850 - RMSE: 528858.7433, Erro percentual médio: 69.36%
        Época 2860 - RMSE: 528448.5727, Erro percentual médio: 69.27%
        Época 2870 - RMSE: 528038.9039, Erro percentual médio: 69.19%
        Época 2880 - RMSE: 527629.7366, Erro percentual médio: 69.10%
        Época 2890 - RMSE: 527221.0703, Erro percentual médio: 69.01%
        Época 2900 - RMSE: 526812.9044, Erro percentual médio: 68.92%
        Época 2910 - RMSE: 526405.2386, Erro percentual médio: 68.83%
        Época 2920 - RMSE: 525998.0723, Erro percentual médio: 68.74%
        Época 2930 - RMSE: 525591.4051, Erro percentual médio: 68.65%
        Época 2940 - RMSE: 525185.2365, Erro percentual médio: 68.57%
        Época 2950 - RMSE: 524779.5661, Erro percentual médio: 68.48%
        Época 2960 - RMSE: 524374.3933, Erro percentual médio: 68.39%
        Época 2970 - RMSE: 523969.7177, Erro percentual médio: 68.30%
        Época 2980 - RMSE: 523565.5389, Erro percentual médio: 68.22%
        Época 2990 - RMSE: 523161.8563, Erro percentual médio: 68.13%
        Época 3000 - RMSE: 522758.6695, Erro percentual médio: 68.04%
        Época 3010 - RMSE: 522355.9781, Erro percentual médio: 67.96%
        Época 3020 - RMSE: 521953.7815, Erro percentual médio: 67.87%
        Época 3030 - RMSE: 521552.0794, Erro percentual médio: 67.79%
        Época 3040 - RMSE: 521150.8712, Erro percentual médio: 67.70%
        Época 3050 - RMSE: 520750.1564, Erro percentual médio: 67.62%
        Época 3060 - RMSE: 520349.9347, Erro percentual médio: 67.53%
        Época 3070 - RMSE: 519950.2055, Erro percentual médio: 67.45%
        Época 3080 - RMSE: 519550.9683, Erro percentual médio: 67.36%
        Época 3090 - RMSE: 519152.2228, Erro percentual médio: 67.28%
        Época 3100 - RMSE: 518753.9685, Erro percentual médio: 67.19%
        Época 3110 - RMSE: 518356.2048, Erro percentual médio: 67.11%
        Época 3120 - RMSE: 517958.9314, Erro percentual médio: 67.02%
        Época 3130 - RMSE: 517562.1477, Erro percentual médio: 66.94%
        Época 3140 - RMSE: 517165.8534, Erro percentual médio: 66.86%
        Época 3150 - RMSE: 516770.0478, Erro percentual médio: 66.77%
        Época 3160 - RMSE: 516374.7307, Erro percentual médio: 66.69%
        Época 3170 - RMSE: 515979.9015, Erro percentual médio: 66.60%
        Época 3180 - RMSE: 515585.5597, Erro percentual médio: 66.52%
        Época 3190 - RMSE: 515191.7049, Erro percentual médio: 66.44%
        Época 3200 - RMSE: 514798.3367, Erro percentual médio: 66.35%
        Época 3210 - RMSE: 514405.4545, Erro percentual médio: 66.27%
        Época 3220 - RMSE: 514013.0580, Erro percentual médio: 66.18%
        Época 3230 - RMSE: 513621.1466, Erro percentual médio: 66.10%
        Época 3240 - RMSE: 513229.7200, Erro percentual médio: 66.02%
        Época 3250 - RMSE: 512838.7776, Erro percentual médio: 65.93%
        Época 3260 - RMSE: 512448.3189, Erro percentual médio: 65.85%
        Época 3270 - RMSE: 512058.3436, Erro percentual médio: 65.77%
        Época 3280 - RMSE: 511668.8512, Erro percentual médio: 65.69%
        Época 3290 - RMSE: 511279.8412, Erro percentual médio: 65.60%
        Época 3300 - RMSE: 510891.3132, Erro percentual médio: 65.52%
        Época 3310 - RMSE: 510503.2667, Erro percentual médio: 65.44%
        Época 3320 - RMSE: 510115.7012, Erro percentual médio: 65.36%
        Época 3330 - RMSE: 509728.6163, Erro percentual médio: 65.28%
        Época 3340 - RMSE: 509342.0116, Erro percentual médio: 65.20%
        Época 3350 - RMSE: 508955.8865, Erro percentual médio: 65.12%
        Época 3360 - RMSE: 508570.2407, Erro percentual médio: 65.04%
        Época 3370 - RMSE: 508185.0737, Erro percentual médio: 64.96%
        Época 3380 - RMSE: 507800.3850, Erro percentual médio: 64.88%
        Época 3390 - RMSE: 507416.1741, Erro percentual médio: 64.80%
        Época 3400 - RMSE: 507032.4407, Erro percentual médio: 64.72%
        Época 3410 - RMSE: 506649.1843, Erro percentual médio: 64.64%
        Época 3420 - RMSE: 506266.4044, Erro percentual médio: 64.56%
        Época 3430 - RMSE: 505884.1005, Erro percentual médio: 64.48%
        Época 3440 - RMSE: 505502.2723, Erro percentual médio: 64.40%
        Época 3450 - RMSE: 505120.9192, Erro percentual médio: 64.32%
        Época 3460 - RMSE: 504740.0409, Erro percentual médio: 64.24%
        Época 3470 - RMSE: 504359.6368, Erro percentual médio: 64.17%
        Época 3480 - RMSE: 503979.7066, Erro percentual médio: 64.09%
        Época 3490 - RMSE: 503600.2497, Erro percentual médio: 64.01%
        Época 3500 - RMSE: 503221.2657, Erro percentual médio: 63.93%
        Época 3510 - RMSE: 502842.7542, Erro percentual médio: 63.85%
        Época 3520 - RMSE: 502464.7147, Erro percentual médio: 63.77%
        Época 3530 - RMSE: 502087.1468, Erro percentual médio: 63.70%
        Época 3540 - RMSE: 501710.0500, Erro percentual médio: 63.62%
        Época 3550 - RMSE: 501333.4239, Erro percentual médio: 63.54%
        Época 3560 - RMSE: 500957.2680, Erro percentual médio: 63.47%
        Época 3570 - RMSE: 500581.5819, Erro percentual médio: 63.39%
        Época 3580 - RMSE: 500206.3652, Erro percentual médio: 63.31%
        Época 3590 - RMSE: 499831.6173, Erro percentual médio: 63.23%
        Época 3600 - RMSE: 499457.3379, Erro percentual médio: 63.16%
        Época 3610 - RMSE: 499083.5265, Erro percentual médio: 63.08%
        Época 3620 - RMSE: 498710.1827, Erro percentual médio: 63.01%
        Época 3630 - RMSE: 498337.3060, Erro percentual médio: 62.93%
        Época 3640 - RMSE: 497964.8959, Erro percentual médio: 62.85%
        Época 3650 - RMSE: 497592.9521, Erro percentual médio: 62.78%
        Época 3660 - RMSE: 497221.4741, Erro percentual médio: 62.70%
        Época 3670 - RMSE: 496850.4614, Erro percentual médio: 62.63%
        Época 3680 - RMSE: 496479.9136, Erro percentual médio: 62.55%
        Época 3690 - RMSE: 496109.8302, Erro percentual médio: 62.48%
        Época 3700 - RMSE: 495740.2109, Erro percentual médio: 62.40%
        Época 3710 - RMSE: 495371.0552, Erro percentual médio: 62.33%
        Época 3720 - RMSE: 495002.3625, Erro percentual médio: 62.25%
        Época 3730 - RMSE: 494634.1326, Erro percentual médio: 62.18%
        Época 3740 - RMSE: 494266.3649, Erro percentual médio: 62.10%
        Época 3750 - RMSE: 493899.0590, Erro percentual médio: 62.03%
        Época 3760 - RMSE: 493532.2145, Erro percentual médio: 61.96%
        Época 3770 - RMSE: 493165.8310, Erro percentual médio: 61.89%
        Época 3780 - RMSE: 492799.9079, Erro percentual médio: 61.81%
        Época 3790 - RMSE: 492434.4449, Erro percentual médio: 61.74%
        Época 3800 - RMSE: 492069.4414, Erro percentual médio: 61.67%
        Época 3810 - RMSE: 491704.8972, Erro percentual médio: 61.60%
        Época 3820 - RMSE: 491340.8117, Erro percentual médio: 61.53%
        Época 3830 - RMSE: 490977.1845, Erro percentual médio: 61.45%
        Época 3840 - RMSE: 490614.0151, Erro percentual médio: 61.38%
        Época 3850 - RMSE: 490251.3032, Erro percentual médio: 61.31%
        Época 3860 - RMSE: 489889.0483, Erro percentual médio: 61.24%
        Época 3870 - RMSE: 489527.2499, Erro percentual médio: 61.17%
        Época 3880 - RMSE: 489165.9077, Erro percentual médio: 61.10%
        Época 3890 - RMSE: 488805.0211, Erro percentual médio: 61.03%
        Época 3900 - RMSE: 488444.5898, Erro percentual médio: 60.96%
        Época 3910 - RMSE: 488084.6132, Erro percentual médio: 60.89%
        Época 3920 - RMSE: 487725.0911, Erro percentual médio: 60.82%
        Época 3930 - RMSE: 487366.0229, Erro percentual médio: 60.75%
        Época 3940 - RMSE: 487007.4082, Erro percentual médio: 60.68%
        Época 3950 - RMSE: 486649.2466, Erro percentual médio: 60.61%
        Época 3960 - RMSE: 486291.5376, Erro percentual médio: 60.55%
        Época 3970 - RMSE: 485934.2809, Erro percentual médio: 60.48%
        Época 3980 - RMSE: 485577.4759, Erro percentual médio: 60.41%
        Época 3990 - RMSE: 485221.1222, Erro percentual médio: 60.34%
        Época 4000 - RMSE: 484865.2195, Erro percentual médio: 60.27%
        Época 4010 - RMSE: 484509.7672, Erro percentual médio: 60.21%
        Época 4020 - RMSE: 484154.7650, Erro percentual médio: 60.14%
        Época 4030 - RMSE: 483800.2124, Erro percentual médio: 60.07%
        Época 4040 - RMSE: 483446.1090, Erro percentual médio: 60.00%
        Época 4050 - RMSE: 483092.4544, Erro percentual médio: 59.94%
        Época 4060 - RMSE: 482739.2481, Erro percentual médio: 59.87%
        Época 4070 - RMSE: 482386.4896, Erro percentual médio: 59.80%
        Época 4080 - RMSE: 482034.1787, Erro percentual médio: 59.74%
        Época 4090 - RMSE: 481682.3148, Erro percentual médio: 59.67%
        Época 4100 - RMSE: 481330.8975, Erro percentual médio: 59.61%
        Época 4110 - RMSE: 480979.9263, Erro percentual médio: 59.54%
        Época 4120 - RMSE: 480629.4010, Erro percentual médio: 59.47%
        Época 4130 - RMSE: 480279.3209, Erro percentual médio: 59.41%
        Época 4140 - RMSE: 479929.6858, Erro percentual médio: 59.34%
        Época 4150 - RMSE: 479580.4951, Erro percentual médio: 59.28%
        Época 4160 - RMSE: 479231.7484, Erro percentual médio: 59.21%
        Época 4170 - RMSE: 478883.4454, Erro percentual médio: 59.15%
        Época 4180 - RMSE: 478535.5856, Erro percentual médio: 59.08%
        Época 4190 - RMSE: 478188.1685, Erro percentual médio: 59.02%
        Época 4200 - RMSE: 477841.1938, Erro percentual médio: 58.95%
        Época 4210 - RMSE: 477494.6610, Erro percentual médio: 58.89%
        Época 4220 - RMSE: 477148.5697, Erro percentual médio: 58.82%
        Época 4230 - RMSE: 476802.9194, Erro percentual médio: 58.76%
        Época 4240 - RMSE: 476457.7098, Erro percentual médio: 58.70%
        Época 4250 - RMSE: 476112.9404, Erro percentual médio: 58.63%
        Época 4260 - RMSE: 475768.6108, Erro percentual médio: 58.57%
        Época 4270 - RMSE: 475424.7206, Erro percentual médio: 58.51%
        Época 4280 - RMSE: 475081.2693, Erro percentual médio: 58.44%
        Época 4290 - RMSE: 474738.2565, Erro percentual médio: 58.38%
        Época 4300 - RMSE: 474395.6818, Erro percentual médio: 58.32%
        Época 4310 - RMSE: 474053.5449, Erro percentual médio: 58.26%
        Época 4320 - RMSE: 473711.8451, Erro percentual médio: 58.20%
        Época 4330 - RMSE: 473370.5822, Erro percentual médio: 58.13%
        Época 4340 - RMSE: 473029.7558, Erro percentual médio: 58.07%
        Época 4350 - RMSE: 472689.3653, Erro percentual médio: 58.01%
        Época 4360 - RMSE: 472349.4104, Erro percentual médio: 57.95%
        Época 4370 - RMSE: 472009.8906, Erro percentual médio: 57.89%
        Época 4380 - RMSE: 471670.8056, Erro percentual médio: 57.83%
        Época 4390 - RMSE: 471332.1549, Erro percentual médio: 57.77%
        Época 4400 - RMSE: 470993.9380, Erro percentual médio: 57.72%
        Época 4410 - RMSE: 470656.1547, Erro percentual médio: 57.66%
        Época 4420 - RMSE: 470318.8044, Erro percentual médio: 57.60%
        Época 4430 - RMSE: 469981.8867, Erro percentual médio: 57.54%
        Época 4440 - RMSE: 469645.4013, Erro percentual médio: 57.48%
        Época 4450 - RMSE: 469309.3477, Erro percentual médio: 57.43%
        Época 4460 - RMSE: 468973.7254, Erro percentual médio: 57.37%
        Época 4470 - RMSE: 468638.5341, Erro percentual médio: 57.31%
        Época 4480 - RMSE: 468303.7734, Erro percentual médio: 57.26%
        Época 4490 - RMSE: 467969.4428, Erro percentual médio: 57.20%
        Época 4500 - RMSE: 467635.5419, Erro percentual médio: 57.14%
        Época 4510 - RMSE: 467302.0703, Erro percentual médio: 57.09%
        Época 4520 - RMSE: 466969.0276, Erro percentual médio: 57.03%
        Época 4530 - RMSE: 466636.4134, Erro percentual médio: 56.98%
        Época 4540 - RMSE: 466304.2272, Erro percentual médio: 56.92%
        Época 4550 - RMSE: 465972.4687, Erro percentual médio: 56.86%
        Época 4560 - RMSE: 465641.1374, Erro percentual médio: 56.81%
        Época 4570 - RMSE: 465310.2329, Erro percentual médio: 56.75%
        Época 4580 - RMSE: 464979.7548, Erro percentual médio: 56.70%
        Época 4590 - RMSE: 464649.7027, Erro percentual médio: 56.64%
        Época 4600 - RMSE: 464320.0761, Erro percentual médio: 56.59%
        Época 4610 - RMSE: 463990.8747, Erro percentual médio: 56.53%
        Época 4620 - RMSE: 463662.0981, Erro percentual médio: 56.48%
        Época 4630 - RMSE: 463333.7457, Erro percentual médio: 56.42%
        Época 4640 - RMSE: 463005.8173, Erro percentual médio: 56.37%
        Época 4650 - RMSE: 462678.3124, Erro percentual médio: 56.31%
        Época 4660 - RMSE: 462351.2306, Erro percentual médio: 56.26%
        Época 4670 - RMSE: 462024.5715, Erro percentual médio: 56.20%
        Época 4680 - RMSE: 461698.3347, Erro percentual médio: 56.15%
        Época 4690 - RMSE: 461372.5197, Erro percentual médio: 56.09%
        Época 4700 - RMSE: 461047.1262, Erro percentual médio: 56.04%
        Época 4710 - RMSE: 460722.1537, Erro percentual médio: 55.99%
        Época 4720 - RMSE: 460397.6019, Erro percentual médio: 55.93%
        Época 4730 - RMSE: 460073.4702, Erro percentual médio: 55.88%
        Época 4740 - RMSE: 459749.7584, Erro percentual médio: 55.83%
        Época 4750 - RMSE: 459426.4660, Erro percentual médio: 55.77%
        Época 4760 - RMSE: 459103.5926, Erro percentual médio: 55.72%
        Época 4770 - RMSE: 458781.1378, Erro percentual médio: 55.67%
        Época 4780 - RMSE: 458459.1011, Erro percentual médio: 55.61%
        Época 4790 - RMSE: 458137.4823, Erro percentual médio: 55.56%
        Época 4800 - RMSE: 457816.2808, Erro percentual médio: 55.51%
        Época 4810 - RMSE: 457495.4962, Erro percentual médio: 55.45%
        Época 4820 - RMSE: 457175.1282, Erro percentual médio: 55.40%
        Época 4830 - RMSE: 456855.1764, Erro percentual médio: 55.35%
        Época 4840 - RMSE: 456535.6402, Erro percentual médio: 55.30%
        Época 4850 - RMSE: 456216.5195, Erro percentual médio: 55.25%
        Época 4860 - RMSE: 455897.8136, Erro percentual médio: 55.19%
        Época 4870 - RMSE: 455579.5223, Erro percentual médio: 55.14%
        Época 4880 - RMSE: 455261.6450, Erro percentual médio: 55.09%
        Época 4890 - RMSE: 454944.1815, Erro percentual médio: 55.04%
        Época 4900 - RMSE: 454627.1313, Erro percentual médio: 54.99%
        Época 4910 - RMSE: 454310.4940, Erro percentual médio: 54.94%
        Época 4920 - RMSE: 453994.2692, Erro percentual médio: 54.89%
        Época 4930 - RMSE: 453678.4565, Erro percentual médio: 54.84%
        Época 4940 - RMSE: 453363.0554, Erro percentual médio: 54.79%
        Época 4950 - RMSE: 453048.0657, Erro percentual médio: 54.74%
        Época 4960 - RMSE: 452733.4868, Erro percentual médio: 54.69%
        Época 4970 - RMSE: 452419.3184, Erro percentual médio: 54.65%
        Época 4980 - RMSE: 452105.5601, Erro percentual médio: 54.60%
        Época 4990 - RMSE: 451792.2115, Erro percentual médio: 54.55%
        Época 5000 - RMSE: 451479.2722, Erro percentual médio: 54.50%
        Época 5010 - RMSE: 451166.7417, Erro percentual médio: 54.45%
        Época 5020 - RMSE: 450854.6197, Erro percentual médio: 54.40%
        Época 5030 - RMSE: 450542.9057, Erro percentual médio: 54.36%
        Época 5040 - RMSE: 450231.5995, Erro percentual médio: 54.31%
        Época 5050 - RMSE: 449920.7005, Erro percentual médio: 54.26%
        Época 5060 - RMSE: 449610.2084, Erro percentual médio: 54.21%
        Época 5070 - RMSE: 449300.1227, Erro percentual médio: 54.17%
        Época 5080 - RMSE: 448990.4431, Erro percentual médio: 54.12%
        Época 5090 - RMSE: 448681.1692, Erro percentual médio: 54.07%
        Época 5100 - RMSE: 448372.3006, Erro percentual médio: 54.02%
        Época 5110 - RMSE: 448063.8368, Erro percentual médio: 53.98%
        Época 5120 - RMSE: 447755.7775, Erro percentual médio: 53.93%
        Época 5130 - RMSE: 447448.1223, Erro percentual médio: 53.88%
        Época 5140 - RMSE: 447140.8707, Erro percentual médio: 53.84%
        Época 5150 - RMSE: 446834.0225, Erro percentual médio: 53.79%
        Época 5160 - RMSE: 446527.5771, Erro percentual médio: 53.75%
        Época 5170 - RMSE: 446221.5342, Erro percentual médio: 53.70%
        Época 5180 - RMSE: 445915.8934, Erro percentual médio: 53.66%
        Época 5190 - RMSE: 445610.6543, Erro percentual médio: 53.61%
        Época 5200 - RMSE: 445305.8164, Erro percentual médio: 53.57%
        Época 5210 - RMSE: 445001.3795, Erro percentual médio: 53.52%
        Época 5220 - RMSE: 444697.3431, Erro percentual médio: 53.48%
        Época 5230 - RMSE: 444393.7068, Erro percentual médio: 53.43%
        Época 5240 - RMSE: 444090.4702, Erro percentual médio: 53.39%
        Época 5250 - RMSE: 443787.6330, Erro percentual médio: 53.34%
        Época 5260 - RMSE: 443485.1946, Erro percentual médio: 53.30%
        Época 5270 - RMSE: 443183.1548, Erro percentual médio: 53.26%
        Época 5280 - RMSE: 442881.5131, Erro percentual médio: 53.21%
        Época 5290 - RMSE: 442580.2692, Erro percentual médio: 53.17%
        Época 5300 - RMSE: 442279.4226, Erro percentual médio: 53.13%
        Época 5310 - RMSE: 441978.9730, Erro percentual médio: 53.09%
        Época 5320 - RMSE: 441678.9199, Erro percentual médio: 53.04%
        Época 5330 - RMSE: 441379.2630, Erro percentual médio: 53.00%
        Época 5340 - RMSE: 441080.0019, Erro percentual médio: 52.96%
        Época 5350 - RMSE: 440781.1361, Erro percentual médio: 52.92%
        Época 5360 - RMSE: 440482.6654, Erro percentual médio: 52.88%
        Época 5370 - RMSE: 440184.5892, Erro percentual médio: 52.83%
        Época 5380 - RMSE: 439886.9073, Erro percentual médio: 52.79%
        Época 5390 - RMSE: 439589.6191, Erro percentual médio: 52.75%
        Época 5400 - RMSE: 439292.7244, Erro percentual médio: 52.71%
        Época 5410 - RMSE: 438996.2228, Erro percentual médio: 52.67%
        Época 5420 - RMSE: 438700.1138, Erro percentual médio: 52.63%
        Época 5430 - RMSE: 438404.3970, Erro percentual médio: 52.59%
        Época 5440 - RMSE: 438109.0721, Erro percentual médio: 52.54%
        Época 5450 - RMSE: 437814.1387, Erro percentual médio: 52.50%
        Época 5460 - RMSE: 437519.5964, Erro percentual médio: 52.46%
        Época 5470 - RMSE: 437225.4447, Erro percentual médio: 52.42%
        Época 5480 - RMSE: 436931.6834, Erro percentual médio: 52.38%
        Época 5490 - RMSE: 436638.3120, Erro percentual médio: 52.34%
        Época 5500 - RMSE: 436345.3301, Erro percentual médio: 52.30%
        Época 5510 - RMSE: 436052.7374, Erro percentual médio: 52.26%
        Época 5520 - RMSE: 435760.5334, Erro percentual médio: 52.22%
        Época 5530 - RMSE: 435468.7178, Erro percentual médio: 52.18%
        Época 5540 - RMSE: 435177.2902, Erro percentual médio: 52.14%
        Época 5550 - RMSE: 434886.2502, Erro percentual médio: 52.10%
        Época 5560 - RMSE: 434595.5973, Erro percentual médio: 52.06%
        Época 5570 - RMSE: 434305.3313, Erro percentual médio: 52.02%
        Época 5580 - RMSE: 434015.4518, Erro percentual médio: 51.98%
        Época 5590 - RMSE: 433725.9583, Erro percentual médio: 51.94%
        Época 5600 - RMSE: 433436.8504, Erro percentual médio: 51.91%
        Época 5610 - RMSE: 433148.1278, Erro percentual médio: 51.87%
        Época 5620 - RMSE: 432859.7901, Erro percentual médio: 51.83%
        Época 5630 - RMSE: 432571.8370, Erro percentual médio: 51.79%
        Época 5640 - RMSE: 432284.2679, Erro percentual médio: 51.75%
        Época 5650 - RMSE: 431997.0826, Erro percentual médio: 51.71%
        Época 5660 - RMSE: 431710.2806, Erro percentual médio: 51.68%
        Época 5670 - RMSE: 431423.8616, Erro percentual médio: 51.64%
        Época 5680 - RMSE: 431137.8252, Erro percentual médio: 51.60%
        Época 5690 - RMSE: 430852.1710, Erro percentual médio: 51.56%
        Época 5700 - RMSE: 430566.8987, Erro percentual médio: 51.52%
        Época 5710 - RMSE: 430282.0077, Erro percentual médio: 51.49%
        Época 5720 - RMSE: 429997.4978, Erro percentual médio: 51.45%
        Época 5730 - RMSE: 429713.3686, Erro percentual médio: 51.41%
        Época 5740 - RMSE: 429429.6197, Erro percentual médio: 51.38%
        Época 5750 - RMSE: 429146.2507, Erro percentual médio: 51.34%
        Época 5760 - RMSE: 428863.2612, Erro percentual médio: 51.31%
        Época 5770 - RMSE: 428580.6508, Erro percentual médio: 51.27%
        Época 5780 - RMSE: 428298.4192, Erro percentual médio: 51.23%
        Época 5790 - RMSE: 428016.5660, Erro percentual médio: 51.20%
        Época 5800 - RMSE: 427735.0908, Erro percentual médio: 51.16%
        Época 5810 - RMSE: 427453.9932, Erro percentual médio: 51.13%
        Época 5820 - RMSE: 427173.2729, Erro percentual médio: 51.09%
        Época 5830 - RMSE: 426892.9294, Erro percentual médio: 51.05%
        Época 5840 - RMSE: 426612.9624, Erro percentual médio: 51.02%
        Época 5850 - RMSE: 426333.3716, Erro percentual médio: 50.98%
        Época 5860 - RMSE: 426054.1564, Erro percentual médio: 50.95%
        Época 5870 - RMSE: 425775.3166, Erro percentual médio: 50.91%
        Época 5880 - RMSE: 425496.8517, Erro percentual médio: 50.88%
        Época 5890 - RMSE: 425218.7614, Erro percentual médio: 50.84%
        Época 5900 - RMSE: 424941.0454, Erro percentual médio: 50.81%
        Época 5910 - RMSE: 424663.7032, Erro percentual médio: 50.77%
        Época 5920 - RMSE: 424386.7344, Erro percentual médio: 50.74%
        Época 5930 - RMSE: 424110.1387, Erro percentual médio: 50.70%
        Época 5940 - RMSE: 423833.9157, Erro percentual médio: 50.67%
        Época 5950 - RMSE: 423558.0650, Erro percentual médio: 50.63%
        Época 5960 - RMSE: 423282.5863, Erro percentual médio: 50.60%
        Época 5970 - RMSE: 423007.4791, Erro percentual médio: 50.56%
        Época 5980 - RMSE: 422732.7432, Erro percentual médio: 50.53%
        Época 5990 - RMSE: 422458.3780, Erro percentual médio: 50.49%
        Época 6000 - RMSE: 422184.3833, Erro percentual médio: 50.46%
        Época 6010 - RMSE: 421910.7587, Erro percentual médio: 50.42%
        Época 6020 - RMSE: 421637.5037, Erro percentual médio: 50.39%
        Época 6030 - RMSE: 421364.6181, Erro percentual médio: 50.35%
        Época 6040 - RMSE: 421092.1014, Erro percentual médio: 50.32%
        Época 6050 - RMSE: 420819.9533, Erro percentual médio: 50.29%
        Época 6060 - RMSE: 420548.1733, Erro percentual médio: 50.25%
        Época 6070 - RMSE: 420276.7612, Erro percentual médio: 50.22%
        Época 6080 - RMSE: 420005.7166, Erro percentual médio: 50.18%
        Época 6090 - RMSE: 419735.0390, Erro percentual médio: 50.15%
        Época 6100 - RMSE: 419464.7281, Erro percentual médio: 50.12%
        Época 6110 - RMSE: 419194.7835, Erro percentual médio: 50.08%
        Época 6120 - RMSE: 418925.2049, Erro percentual médio: 50.05%
        Época 6130 - RMSE: 418655.9918, Erro percentual médio: 50.02%
        Época 6140 - RMSE: 418387.1440, Erro percentual médio: 49.99%
        Época 6150 - RMSE: 418118.6610, Erro percentual médio: 49.95%
        Época 6160 - RMSE: 417850.5424, Erro percentual médio: 49.92%
        Época 6170 - RMSE: 417582.7880, Erro percentual médio: 49.89%
        Época 6180 - RMSE: 417315.3973, Erro percentual médio: 49.86%
        Época 6190 - RMSE: 417048.3699, Erro percentual médio: 49.82%
        Época 6200 - RMSE: 416781.7055, Erro percentual médio: 49.79%
        Época 6210 - RMSE: 416515.4037, Erro percentual médio: 49.76%
        Época 6220 - RMSE: 416249.4642, Erro percentual médio: 49.73%
        Época 6230 - RMSE: 415983.8865, Erro percentual médio: 49.70%
        Época 6240 - RMSE: 415718.6703, Erro percentual médio: 49.66%
        Época 6250 - RMSE: 415453.8152, Erro percentual médio: 49.63%
        Época 6260 - RMSE: 415189.3209, Erro percentual médio: 49.60%
        Época 6270 - RMSE: 414925.1870, Erro percentual médio: 49.57%
        Época 6280 - RMSE: 414661.4131, Erro percentual médio: 49.54%
        Época 6290 - RMSE: 414397.9989, Erro percentual médio: 49.51%
        Época 6300 - RMSE: 414134.9440, Erro percentual médio: 49.48%
        Época 6310 - RMSE: 413872.2479, Erro percentual médio: 49.44%
        Época 6320 - RMSE: 413609.9104, Erro percentual médio: 49.41%
        Época 6330 - RMSE: 413347.9311, Erro percentual médio: 49.38%
        Época 6340 - RMSE: 413086.3096, Erro percentual médio: 49.35%
        Época 6350 - RMSE: 412825.0455, Erro percentual médio: 49.32%
        Época 6360 - RMSE: 412564.1386, Erro percentual médio: 49.29%
        Época 6370 - RMSE: 412303.5883, Erro percentual médio: 49.26%
        Época 6380 - RMSE: 412043.3944, Erro percentual médio: 49.23%
        Época 6390 - RMSE: 411783.5564, Erro percentual médio: 49.20%
        Época 6400 - RMSE: 411524.0741, Erro percentual médio: 49.17%
        Época 6410 - RMSE: 411264.9470, Erro percentual médio: 49.14%
        Época 6420 - RMSE: 411006.1748, Erro percentual médio: 49.11%
        Época 6430 - RMSE: 410747.7571, Erro percentual médio: 49.08%
        Época 6440 - RMSE: 410489.6935, Erro percentual médio: 49.05%
        Época 6450 - RMSE: 410231.9838, Erro percentual médio: 49.02%
        Época 6460 - RMSE: 409974.6274, Erro percentual médio: 48.99%
        Época 6470 - RMSE: 409717.6241, Erro percentual médio: 48.96%
        Época 6480 - RMSE: 409460.9735, Erro percentual médio: 48.93%
        Época 6490 - RMSE: 409204.6753, Erro percentual médio: 48.90%
        Época 6500 - RMSE: 408948.7289, Erro percentual médio: 48.87%
        Época 6510 - RMSE: 408693.1342, Erro percentual médio: 48.84%
        Época 6520 - RMSE: 408437.8908, Erro percentual médio: 48.81%
        Época 6530 - RMSE: 408182.9982, Erro percentual médio: 48.78%
        Época 6540 - RMSE: 407928.4561, Erro percentual médio: 48.76%
        Época 6550 - RMSE: 407674.2641, Erro percentual médio: 48.73%
        Época 6560 - RMSE: 407420.4219, Erro percentual médio: 48.70%
        Época 6570 - RMSE: 407166.9292, Erro percentual médio: 48.67%
        Época 6580 - RMSE: 406913.7855, Erro percentual médio: 48.64%
        Época 6590 - RMSE: 406660.9905, Erro percentual médio: 48.61%
        Época 6600 - RMSE: 406408.5439, Erro percentual médio: 48.58%
        Época 6610 - RMSE: 406156.4452, Erro percentual médio: 48.56%
        Época 6620 - RMSE: 405904.6942, Erro percentual médio: 48.53%
        Época 6630 - RMSE: 405653.2904, Erro percentual médio: 48.50%
        Época 6640 - RMSE: 405402.2335, Erro percentual médio: 48.47%
        Época 6650 - RMSE: 405151.5231, Erro percentual médio: 48.44%
        Época 6660 - RMSE: 404901.1589, Erro percentual médio: 48.42%
        Época 6670 - RMSE: 404651.1405, Erro percentual médio: 48.39%
        Época 6680 - RMSE: 404401.4675, Erro percentual médio: 48.36%
        Época 6690 - RMSE: 404152.1397, Erro percentual médio: 48.33%
        Época 6700 - RMSE: 403903.1566, Erro percentual médio: 48.31%
        Época 6710 - RMSE: 403654.5179, Erro percentual médio: 48.28%
        Época 6720 - RMSE: 403406.2231, Erro percentual médio: 48.25%
        Época 6730 - RMSE: 403158.2721, Erro percentual médio: 48.23%
        Época 6740 - RMSE: 402910.6643, Erro percentual médio: 48.20%
        Época 6750 - RMSE: 402663.3995, Erro percentual médio: 48.17%
        Época 6760 - RMSE: 402416.4773, Erro percentual médio: 48.15%
        Época 6770 - RMSE: 402169.8972, Erro percentual médio: 48.12%
        Época 6780 - RMSE: 401923.6591, Erro percentual médio: 48.09%
        Época 6790 - RMSE: 401677.7625, Erro percentual médio: 48.07%
        Época 6800 - RMSE: 401432.2070, Erro percentual médio: 48.04%
        Época 6810 - RMSE: 401186.9923, Erro percentual médio: 48.01%
        Época 6820 - RMSE: 400942.1181, Erro percentual médio: 47.99%
        Época 6830 - RMSE: 400697.5839, Erro percentual médio: 47.96%
        Época 6840 - RMSE: 400453.3895, Erro percentual médio: 47.93%
        Época 6850 - RMSE: 400209.5344, Erro percentual médio: 47.91%
        Época 6860 - RMSE: 399966.0184, Erro percentual médio: 47.88%
        Época 6870 - RMSE: 399722.8410, Erro percentual médio: 47.86%
        Época 6880 - RMSE: 399480.0019, Erro percentual médio: 47.83%
        Época 6890 - RMSE: 399237.5008, Erro percentual médio: 47.81%
        Época 6900 - RMSE: 398995.3373, Erro percentual médio: 47.78%
        Época 6910 - RMSE: 398753.5110, Erro percentual médio: 47.76%
        Época 6920 - RMSE: 398512.0216, Erro percentual médio: 47.73%
        Época 6930 - RMSE: 398270.8687, Erro percentual médio: 47.71%
        Época 6940 - RMSE: 398030.0520, Erro percentual médio: 47.68%
        Época 6950 - RMSE: 397789.5712, Erro percentual médio: 47.66%
        Época 6960 - RMSE: 397549.4258, Erro percentual médio: 47.63%
        Época 6970 - RMSE: 397309.6155, Erro percentual médio: 47.61%
        Época 6980 - RMSE: 397070.1400, Erro percentual médio: 47.59%
        Época 6990 - RMSE: 396830.9989, Erro percentual médio: 47.56%
        Época 7000 - RMSE: 396592.1919, Erro percentual médio: 47.54%
        Época 7010 - RMSE: 396353.7186, Erro percentual médio: 47.51%
        Época 7020 - RMSE: 396115.5786, Erro percentual médio: 47.49%
        Época 7030 - RMSE: 395877.7717, Erro percentual médio: 47.47%
        Época 7040 - RMSE: 395640.2973, Erro percentual médio: 47.44%
        Época 7050 - RMSE: 395403.1553, Erro percentual médio: 47.42%
        Época 7060 - RMSE: 395166.3453, Erro percentual médio: 47.39%
        Época 7070 - RMSE: 394929.8668, Erro percentual médio: 47.37%
        Época 7080 - RMSE: 394693.7196, Erro percentual médio: 47.35%
        Época 7090 - RMSE: 394457.9033, Erro percentual médio: 47.32%
        Época 7100 - RMSE: 394222.4175, Erro percentual médio: 47.30%
        Época 7110 - RMSE: 393987.2619, Erro percentual médio: 47.28%
        Época 7120 - RMSE: 393752.4362, Erro percentual médio: 47.25%
        Época 7130 - RMSE: 393517.9400, Erro percentual médio: 47.23%
        Época 7140 - RMSE: 393283.7729, Erro percentual médio: 47.21%
        Época 7150 - RMSE: 393049.9346, Erro percentual médio: 47.18%
        Época 7160 - RMSE: 392816.4248, Erro percentual médio: 47.16%
        Época 7170 - RMSE: 392583.2430, Erro percentual médio: 47.14%
        Época 7180 - RMSE: 392350.3890, Erro percentual médio: 47.12%
        Época 7190 - RMSE: 392117.8625, Erro percentual médio: 47.09%
        Época 7200 - RMSE: 391885.6629, Erro percentual médio: 47.07%
        Época 7210 - RMSE: 391653.7901, Erro percentual médio: 47.05%
        Época 7220 - RMSE: 391422.2437, Erro percentual médio: 47.03%
        Época 7230 - RMSE: 391191.0232, Erro percentual médio: 47.00%
        Época 7240 - RMSE: 390960.1284, Erro percentual médio: 46.98%
        Época 7250 - RMSE: 390729.5589, Erro percentual médio: 46.96%
        Época 7260 - RMSE: 390499.3144, Erro percentual médio: 46.94%
        Época 7270 - RMSE: 390269.3946, Erro percentual médio: 46.92%
        Época 7280 - RMSE: 390039.7990, Erro percentual médio: 46.90%
        Época 7290 - RMSE: 389810.5273, Erro percentual médio: 46.87%
        Época 7300 - RMSE: 389581.5792, Erro percentual médio: 46.85%
        Época 7310 - RMSE: 389352.9543, Erro percentual médio: 46.83%
        Época 7320 - RMSE: 389124.6524, Erro percentual médio: 46.81%
        Época 7330 - RMSE: 388896.6730, Erro percentual médio: 46.79%
        Época 7340 - RMSE: 388669.0158, Erro percentual médio: 46.77%
        Época 7350 - RMSE: 388441.6804, Erro percentual médio: 46.74%
        Época 7360 - RMSE: 388214.6666, Erro percentual médio: 46.72%
        Época 7370 - RMSE: 387987.9739, Erro percentual médio: 46.70%
        Época 7380 - RMSE: 387761.6020, Erro percentual médio: 46.68%
        Época 7390 - RMSE: 387535.5507, Erro percentual médio: 46.66%
        Época 7400 - RMSE: 387309.8194, Erro percentual médio: 46.64%
        Época 7410 - RMSE: 387084.4080, Erro percentual médio: 46.62%
        Época 7420 - RMSE: 386859.3160, Erro percentual médio: 46.60%
        Época 7430 - RMSE: 386634.5431, Erro percentual médio: 46.58%
        Época 7440 - RMSE: 386410.0890, Erro percentual médio: 46.56%
        Época 7450 - RMSE: 386185.9533, Erro percentual médio: 46.54%
        Época 7460 - RMSE: 385962.1356, Erro percentual médio: 46.52%
        Época 7470 - RMSE: 385738.6357, Erro percentual médio: 46.50%
        Época 7480 - RMSE: 385515.4532, Erro percentual médio: 46.48%
        Época 7490 - RMSE: 385292.5877, Erro percentual médio: 46.46%
        Época 7500 - RMSE: 385070.0390, Erro percentual médio: 46.44%
        Época 7510 - RMSE: 384847.8066, Erro percentual médio: 46.42%
        Época 7520 - RMSE: 384625.8902, Erro percentual médio: 46.40%
        Época 7530 - RMSE: 384404.2895, Erro percentual médio: 46.38%
        Época 7540 - RMSE: 384183.0042, Erro percentual médio: 46.36%
        Época 7550 - RMSE: 383962.0339, Erro percentual médio: 46.34%
        Época 7560 - RMSE: 383741.3782, Erro percentual médio: 46.32%
        Época 7570 - RMSE: 383521.0368, Erro percentual médio: 46.30%
        Época 7580 - RMSE: 383301.0094, Erro percentual médio: 46.28%
        Época 7590 - RMSE: 383081.2957, Erro percentual médio: 46.26%
        Época 7600 - RMSE: 382861.8953, Erro percentual médio: 46.24%
        Época 7610 - RMSE: 382642.8078, Erro percentual médio: 46.22%
        Época 7620 - RMSE: 382424.0329, Erro percentual médio: 46.20%
        Época 7630 - RMSE: 382205.5703, Erro percentual médio: 46.19%
        Época 7640 - RMSE: 381987.4197, Erro percentual médio: 46.17%
        Época 7650 - RMSE: 381769.5807, Erro percentual médio: 46.15%
        Época 7660 - RMSE: 381552.0529, Erro percentual médio: 46.13%
        Época 7670 - RMSE: 381334.8361, Erro percentual médio: 46.11%
        Época 7680 - RMSE: 381117.9298, Erro percentual médio: 46.09%
        Época 7690 - RMSE: 380901.3339, Erro percentual médio: 46.07%
        Época 7700 - RMSE: 380685.0478, Erro percentual médio: 46.05%
        Época 7710 - RMSE: 380469.0713, Erro percentual médio: 46.04%
        Época 7720 - RMSE: 380253.4040, Erro percentual médio: 46.02%
        Época 7730 - RMSE: 380038.0457, Erro percentual médio: 46.00%
        Época 7740 - RMSE: 379822.9959, Erro percentual médio: 45.98%
        Época 7750 - RMSE: 379608.2543, Erro percentual médio: 45.96%
        Época 7760 - RMSE: 379393.8207, Erro percentual médio: 45.94%
        Época 7770 - RMSE: 379179.6945, Erro percentual médio: 45.92%
        Época 7780 - RMSE: 378965.8757, Erro percentual médio: 45.91%
        Época 7790 - RMSE: 378752.3637, Erro percentual médio: 45.89%
        Época 7800 - RMSE: 378539.1582, Erro percentual médio: 45.87%
        Época 7810 - RMSE: 378326.2590, Erro percentual médio: 45.85%
        Época 7820 - RMSE: 378113.6656, Erro percentual médio: 45.83%
        Época 7830 - RMSE: 377901.3778, Erro percentual médio: 45.81%
        Época 7840 - RMSE: 377689.3952, Erro percentual médio: 45.80%
        Época 7850 - RMSE: 377477.7174, Erro percentual médio: 45.78%
        Época 7860 - RMSE: 377266.3442, Erro percentual médio: 45.76%
        Época 7870 - RMSE: 377055.2753, Erro percentual médio: 45.74%
        Época 7880 - RMSE: 376844.5101, Erro percentual médio: 45.72%
        Época 7890 - RMSE: 376634.0486, Erro percentual médio: 45.71%
        Época 7900 - RMSE: 376423.8902, Erro percentual médio: 45.69%
        Época 7910 - RMSE: 376214.0347, Erro percentual médio: 45.67%
        Época 7920 - RMSE: 376004.4817, Erro percentual médio: 45.65%
        Época 7930 - RMSE: 375795.2310, Erro percentual médio: 45.64%
        Época 7940 - RMSE: 375586.2821, Erro percentual médio: 45.62%
        Época 7950 - RMSE: 375377.6348, Erro percentual médio: 45.60%
        Época 7960 - RMSE: 375169.2886, Erro percentual médio: 45.58%
        Época 7970 - RMSE: 374961.2434, Erro percentual médio: 45.57%
        Época 7980 - RMSE: 374753.4987, Erro percentual médio: 45.55%
        Época 7990 - RMSE: 374546.0542, Erro percentual médio: 45.53%
        Época 8000 - RMSE: 374338.9096, Erro percentual médio: 45.52%
        Época 8010 - RMSE: 374132.0645, Erro percentual médio: 45.50%
        Época 8020 - RMSE: 373925.5187, Erro percentual médio: 45.48%
        Época 8030 - RMSE: 373719.2717, Erro percentual médio: 45.47%
        Época 8040 - RMSE: 373513.3233, Erro percentual médio: 45.45%
        Época 8050 - RMSE: 373307.6731, Erro percentual médio: 45.43%
        Época 8060 - RMSE: 373102.3209, Erro percentual médio: 45.42%
        Época 8070 - RMSE: 372897.2661, Erro percentual médio: 45.40%
        Época 8080 - RMSE: 372692.5087, Erro percentual médio: 45.38%
        Época 8090 - RMSE: 372488.0481, Erro percentual médio: 45.37%
        Época 8100 - RMSE: 372283.8841, Erro percentual médio: 45.35%
        Época 8110 - RMSE: 372080.0164, Erro percentual médio: 45.34%
        Época 8120 - RMSE: 371876.4446, Erro percentual médio: 45.32%
        Época 8130 - RMSE: 371673.1684, Erro percentual médio: 45.30%
        Época 8140 - RMSE: 371470.1874, Erro percentual médio: 45.29%
        Época 8150 - RMSE: 371267.5014, Erro percentual médio: 45.27%
        Época 8160 - RMSE: 371065.1099, Erro percentual médio: 45.26%
        Época 8170 - RMSE: 370863.0128, Erro percentual médio: 45.24%
        Época 8180 - RMSE: 370661.2096, Erro percentual médio: 45.23%
        Época 8190 - RMSE: 370459.7000, Erro percentual médio: 45.22%
        Época 8200 - RMSE: 370258.4838, Erro percentual médio: 45.20%
        Época 8210 - RMSE: 370057.5605, Erro percentual médio: 45.19%
        Época 8220 - RMSE: 369856.9298, Erro percentual médio: 45.17%
        Época 8230 - RMSE: 369656.5914, Erro percentual médio: 45.16%
        Época 8240 - RMSE: 369456.5451, Erro percentual médio: 45.14%
        Época 8250 - RMSE: 369256.7904, Erro percentual médio: 45.13%
        Época 8260 - RMSE: 369057.3270, Erro percentual médio: 45.11%
        Época 8270 - RMSE: 368858.1547, Erro percentual médio: 45.10%
        Época 8280 - RMSE: 368659.2730, Erro percentual médio: 45.09%
        Época 8290 - RMSE: 368460.6817, Erro percentual médio: 45.07%
        Época 8300 - RMSE: 368262.3804, Erro percentual médio: 45.06%
        Época 8310 - RMSE: 368064.3688, Erro percentual médio: 45.05%
        Época 8320 - RMSE: 367866.6466, Erro percentual médio: 45.03%
        Época 8330 - RMSE: 367669.2135, Erro percentual médio: 45.02%
        Época 8340 - RMSE: 367472.0691, Erro percentual médio: 45.01%
        Época 8350 - RMSE: 367275.2131, Erro percentual médio: 44.99%
        Época 8360 - RMSE: 367078.6452, Erro percentual médio: 44.98%
        Época 8370 - RMSE: 366882.3650, Erro percentual médio: 44.97%
        Época 8380 - RMSE: 366686.3723, Erro percentual médio: 44.95%
        Época 8390 - RMSE: 366490.6667, Erro percentual médio: 44.94%
        Época 8400 - RMSE: 366295.2479, Erro percentual médio: 44.93%
        Época 8410 - RMSE: 366100.1156, Erro percentual médio: 44.91%
        Época 8420 - RMSE: 365905.2694, Erro percentual médio: 44.90%
        Época 8430 - RMSE: 365710.7090, Erro percentual médio: 44.89%
        Época 8440 - RMSE: 365516.4342, Erro percentual médio: 44.87%
        Época 8450 - RMSE: 365322.4445, Erro percentual médio: 44.86%
        Época 8460 - RMSE: 365128.7396, Erro percentual médio: 44.85%
        Época 8470 - RMSE: 364935.3193, Erro percentual médio: 44.84%
        Época 8480 - RMSE: 364742.1832, Erro percentual médio: 44.82%
        Época 8490 - RMSE: 364549.3310, Erro percentual médio: 44.81%
        Época 8500 - RMSE: 364356.7624, Erro percentual médio: 44.80%
        Época 8510 - RMSE: 364164.4770, Erro percentual médio: 44.79%
        Época 8520 - RMSE: 363972.4745, Erro percentual médio: 44.77%
        Época 8530 - RMSE: 363780.7547, Erro percentual médio: 44.76%
        Época 8540 - RMSE: 363589.3171, Erro percentual médio: 44.75%
        Época 8550 - RMSE: 363398.1615, Erro percentual médio: 44.74%
        Época 8560 - RMSE: 363207.2875, Erro percentual médio: 44.72%
        Época 8570 - RMSE: 363016.6949, Erro percentual médio: 44.71%
        Época 8580 - RMSE: 362826.3833, Erro percentual médio: 44.70%
        Época 8590 - RMSE: 362636.3523, Erro percentual médio: 44.69%
        Época 8600 - RMSE: 362446.6017, Erro percentual médio: 44.68%
        Época 8610 - RMSE: 362257.1312, Erro percentual médio: 44.66%
        Época 8620 - RMSE: 362067.9404, Erro percentual médio: 44.65%
        Época 8630 - RMSE: 361879.0290, Erro percentual médio: 44.64%
        Época 8640 - RMSE: 361690.3967, Erro percentual médio: 44.63%
        Época 8650 - RMSE: 361502.0431, Erro percentual médio: 44.62%
        Época 8660 - RMSE: 361313.9681, Erro percentual médio: 44.61%
        Época 8670 - RMSE: 361126.1711, Erro percentual médio: 44.60%
        Época 8680 - RMSE: 360938.6520, Erro percentual médio: 44.59%
        Época 8690 - RMSE: 360751.4104, Erro percentual médio: 44.58%
        Época 8700 - RMSE: 360564.4459, Erro percentual médio: 44.56%
        Época 8710 - RMSE: 360377.7584, Erro percentual médio: 44.55%
        Época 8720 - RMSE: 360191.3473, Erro percentual médio: 44.54%
        Época 8730 - RMSE: 360005.2125, Erro percentual médio: 44.53%
        Época 8740 - RMSE: 359819.3537, Erro percentual médio: 44.52%
        Época 8750 - RMSE: 359633.7704, Erro percentual médio: 44.51%
        Época 8760 - RMSE: 359448.4624, Erro percentual médio: 44.50%
        Época 8770 - RMSE: 359263.4294, Erro percentual médio: 44.49%
        Época 8780 - RMSE: 359078.6710, Erro percentual médio: 44.48%
        Época 8790 - RMSE: 358894.1870, Erro percentual médio: 44.47%
        Época 8800 - RMSE: 358709.9770, Erro percentual médio: 44.46%
        Época 8810 - RMSE: 358526.0407, Erro percentual médio: 44.45%
        Época 8820 - RMSE: 358342.3778, Erro percentual médio: 44.44%
        Época 8830 - RMSE: 358158.9880, Erro percentual médio: 44.43%
        Época 8840 - RMSE: 357975.8709, Erro percentual médio: 44.42%
        Época 8850 - RMSE: 357793.0263, Erro percentual médio: 44.41%
        Época 8860 - RMSE: 357610.4538, Erro percentual médio: 44.40%
        Época 8870 - RMSE: 357428.1531, Erro percentual médio: 44.39%
        Época 8880 - RMSE: 357246.1240, Erro percentual médio: 44.37%
        Época 8890 - RMSE: 357064.3660, Erro percentual médio: 44.36%
        Época 8900 - RMSE: 356882.8789, Erro percentual médio: 44.35%
        Época 8910 - RMSE: 356701.6624, Erro percentual médio: 44.34%
        Época 8920 - RMSE: 356520.7162, Erro percentual médio: 44.33%
        Época 8930 - RMSE: 356340.0398, Erro percentual médio: 44.32%
        Época 8940 - RMSE: 356159.6331, Erro percentual médio: 44.31%
        Época 8950 - RMSE: 355979.4958, Erro percentual médio: 44.30%
        Época 8960 - RMSE: 355799.6274, Erro percentual médio: 44.29%
        Época 8970 - RMSE: 355620.0277, Erro percentual médio: 44.28%
        Época 8980 - RMSE: 355440.6964, Erro percentual médio: 44.27%
        Época 8990 - RMSE: 355261.6332, Erro percentual médio: 44.26%
        Época 9000 - RMSE: 355082.8377, Erro percentual médio: 44.25%
        Época 9010 - RMSE: 354904.3097, Erro percentual médio: 44.24%
        Época 9020 - RMSE: 354726.0488, Erro percentual médio: 44.24%
        Época 9030 - RMSE: 354548.0547, Erro percentual médio: 44.23%
        Época 9040 - RMSE: 354370.3272, Erro percentual médio: 44.22%
        Época 9050 - RMSE: 354192.8658, Erro percentual médio: 44.21%
        Época 9060 - RMSE: 354015.6704, Erro percentual médio: 44.20%
        Época 9070 - RMSE: 353838.7405, Erro percentual médio: 44.19%
        Época 9080 - RMSE: 353662.0759, Erro percentual médio: 44.18%
        Época 9090 - RMSE: 353485.6763, Erro percentual médio: 44.17%
        Época 9100 - RMSE: 353309.5413, Erro percentual médio: 44.16%
        Época 9110 - RMSE: 353133.6707, Erro percentual médio: 44.15%
        Época 9120 - RMSE: 352958.0641, Erro percentual médio: 44.14%
        Época 9130 - RMSE: 352782.7213, Erro percentual médio: 44.13%
        Época 9140 - RMSE: 352607.6418, Erro percentual médio: 44.12%
        Época 9150 - RMSE: 352432.8255, Erro percentual médio: 44.12%
        Época 9160 - RMSE: 352258.2720, Erro percentual médio: 44.11%
        Época 9170 - RMSE: 352083.9810, Erro percentual médio: 44.10%
        Época 9180 - RMSE: 351909.9521, Erro percentual médio: 44.09%
        Época 9190 - RMSE: 351736.1852, Erro percentual médio: 44.08%
        Época 9200 - RMSE: 351562.6798, Erro percentual médio: 44.07%
        Época 9210 - RMSE: 351389.4357, Erro percentual médio: 44.06%
        Época 9220 - RMSE: 351216.4525, Erro percentual médio: 44.06%
        Época 9230 - RMSE: 351043.7300, Erro percentual médio: 44.05%
        Época 9240 - RMSE: 350871.2678, Erro percentual médio: 44.04%
        Época 9250 - RMSE: 350699.0657, Erro percentual médio: 44.03%
        Época 9260 - RMSE: 350527.1233, Erro percentual médio: 44.02%
        Época 9270 - RMSE: 350355.4404, Erro percentual médio: 44.02%
        Época 9280 - RMSE: 350184.0165, Erro percentual médio: 44.01%
        Época 9290 - RMSE: 350012.8515, Erro percentual médio: 44.00%
        Época 9300 - RMSE: 349841.9450, Erro percentual médio: 43.99%
        Época 9310 - RMSE: 349671.2967, Erro percentual médio: 43.99%
        Época 9320 - RMSE: 349500.9063, Erro percentual médio: 43.98%
        Época 9330 - RMSE: 349330.7735, Erro percentual médio: 43.97%
        Época 9340 - RMSE: 349160.8980, Erro percentual médio: 43.96%
        Época 9350 - RMSE: 348991.2795, Erro percentual médio: 43.95%
        Época 9360 - RMSE: 348821.9176, Erro percentual médio: 43.95%
        Época 9370 - RMSE: 348652.8122, Erro percentual médio: 43.94%
        Época 9380 - RMSE: 348483.9628, Erro percentual médio: 43.93%
        Época 9390 - RMSE: 348315.3691, Erro percentual médio: 43.92%
        Época 9400 - RMSE: 348147.0310, Erro percentual médio: 43.92%
        Época 9410 - RMSE: 347978.9480, Erro percentual médio: 43.91%
        Época 9420 - RMSE: 347811.1199, Erro percentual médio: 43.90%
        Época 9430 - RMSE: 347643.5463, Erro percentual médio: 43.89%
        Época 9440 - RMSE: 347476.2270, Erro percentual médio: 43.89%
        Época 9450 - RMSE: 347309.1617, Erro percentual médio: 43.88%
        Época 9460 - RMSE: 347142.3500, Erro percentual médio: 43.87%
        Época 9470 - RMSE: 346975.7916, Erro percentual médio: 43.86%
        Época 9480 - RMSE: 346809.4863, Erro percentual médio: 43.86%
        Época 9490 - RMSE: 346643.4338, Erro percentual médio: 43.85%
        Época 9500 - RMSE: 346477.6337, Erro percentual médio: 43.84%
        Época 9510 - RMSE: 346312.0857, Erro percentual médio: 43.84%
        Época 9520 - RMSE: 346146.7896, Erro percentual médio: 43.83%
        Época 9530 - RMSE: 345981.7450, Erro percentual médio: 43.82%
        Época 9540 - RMSE: 345816.9517, Erro percentual médio: 43.82%
        Época 9550 - RMSE: 345652.4093, Erro percentual médio: 43.81%
        Época 9560 - RMSE: 345488.1176, Erro percentual médio: 43.80%
        Época 9570 - RMSE: 345324.0762, Erro percentual médio: 43.80%
        Época 9580 - RMSE: 345160.2848, Erro percentual médio: 43.79%
        Época 9590 - RMSE: 344996.7432, Erro percentual médio: 43.78%
        Época 9600 - RMSE: 344833.4510, Erro percentual médio: 43.78%
        Época 9610 - RMSE: 344670.4080, Erro percentual médio: 43.77%
        Época 9620 - RMSE: 344507.6138, Erro percentual médio: 43.76%
        Época 9630 - RMSE: 344345.0682, Erro percentual médio: 43.76%
        Época 9640 - RMSE: 344182.7708, Erro percentual médio: 43.75%
        Época 9650 - RMSE: 344020.7213, Erro percentual médio: 43.74%
        Época 9660 - RMSE: 343858.9195, Erro percentual médio: 43.74%
        Época 9670 - RMSE: 343697.3651, Erro percentual médio: 43.73%
        Época 9680 - RMSE: 343536.0577, Erro percentual médio: 43.72%
        Época 9690 - RMSE: 343374.9971, Erro percentual médio: 43.72%
        Época 9700 - RMSE: 343214.1829, Erro percentual médio: 43.71%
        Época 9710 - RMSE: 343053.6149, Erro percentual médio: 43.71%
        Época 9720 - RMSE: 342893.2928, Erro percentual médio: 43.70%
        Época 9730 - RMSE: 342733.2162, Erro percentual médio: 43.69%
        Época 9740 - RMSE: 342573.3849, Erro percentual médio: 43.69%
        Época 9750 - RMSE: 342413.7986, Erro percentual médio: 43.68%
        Época 9760 - RMSE: 342254.4570, Erro percentual médio: 43.67%
        Época 9770 - RMSE: 342095.3597, Erro percentual médio: 43.67%
        Época 9780 - RMSE: 341936.5066, Erro percentual médio: 43.66%
        Época 9790 - RMSE: 341777.8972, Erro percentual médio: 43.66%
        Época 9800 - RMSE: 341619.5313, Erro percentual médio: 43.65%
        Época 9810 - RMSE: 341461.4087, Erro percentual médio: 43.64%
        Época 9820 - RMSE: 341303.5289, Erro percentual médio: 43.64%
        Época 9830 - RMSE: 341145.8918, Erro percentual médio: 43.63%
        Época 9840 - RMSE: 340988.4969, Erro percentual médio: 43.63%
        Época 9850 - RMSE: 340831.3441, Erro percentual médio: 43.62%
        Época 9860 - RMSE: 340674.4331, Erro percentual médio: 43.61%
        Época 9870 - RMSE: 340517.7634, Erro percentual médio: 43.61%
        Época 9880 - RMSE: 340361.3349, Erro percentual médio: 43.60%
        Época 9890 - RMSE: 340205.1473, Erro percentual médio: 43.60%
        Época 9900 - RMSE: 340049.2002, Erro percentual médio: 43.59%
        Época 9910 - RMSE: 339893.4934, Erro percentual médio: 43.59%
        Época 9920 - RMSE: 339738.0265, Erro percentual médio: 43.58%
        Época 9930 - RMSE: 339582.7993, Erro percentual médio: 43.57%
        Época 9940 - RMSE: 339427.8115, Erro percentual médio: 43.57%
        Época 9950 - RMSE: 339273.0628, Erro percentual médio: 43.56%
        Época 9960 - RMSE: 339118.5529, Erro percentual médio: 43.56%
        Época 9970 - RMSE: 338964.2815, Erro percentual médio: 43.55%
        Época 9980 - RMSE: 338810.2483, Erro percentual médio: 43.54%
        Época 9990 - RMSE: 338656.4531, Erro percentual médio: 43.54%
        Época 9999 - RMSE: 338518.2405, Erro percentual médio: 43.53%
        RMSE: 355954.9367
        Erro percentual médio: 48.05%